# Gathing the foundational dataset that this project will be based on

What: Granted Design patents from the USPTO that were applied during the years 1980-2015

How: Using the PatentsView API, which already disambiguates and aggregates patent data for us. This is a product directly from USPTO, makig it a realiable source. 

Data fields we are interested in:
1. patent number
2. application year
3. number of inventors
4. number of assignees
5. number of cited design patents
6. number of cited utility patents
7. number of cited non-patent prior arts
8. assignee name
9. assignee city
10. assignee state
11. assignee country
12. grant year
13. priority date
14. if cited any foreign patents
15. design patent class
16. design patent subclass
17. number of figures
18. if US inventor
19. if any missing citations

In [2]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:

#This particular API doesn't quite follow the standard REST protocol
#This is setting up the parts of the URL that won't change between patents
#See http://www.patentsview.org/api/query-language.html for more info


base_url = "http://www.patentsview.org/api/patents/query?"
field_list = "&f=[\"patent_number\",\"app_date\",\"inventor_id\",\"assignee_city\",\"assignee_country\",\"assignee_state\",\"assignee_organization\",\"assignee_first_name\",\"assignee_last_name\",\"patent_year\",\"uspc_subclass_id\",\"uspc_mainclass_id\",\"cited_patent_number\",\"patent_num_foreign_citations\",\"patent_num_us_patent_citations\",\"forprior_country\", \"forprior_date\", \"patent_firstnamed_inventor_country\"]"

top_level = pd.DataFrame()
applications_level = pd.DataFrame()
assignee_level = pd.DataFrame()
cited_patents_level = pd.DataFrame()
inventor_level = pd.DataFrame()
foreign_priority_level = pd.DataFrame()
uspcs_level = pd.DataFrame()

In [ ]:
#Due to the limits of the amount of results the API can return, query must be performed in loop by date and page

for year in range(1980,2016):
    page = 1
    more_patents = True
    
    while more_patents:
        
        query = "q={{\"_and\":[{{\"patent_type\":\"Design\"}},{{\"_gte\":{{\"app_date\":\"{0}-01-01\"}}}},{{\"_lte\":{{\"app_date\":\"{0}-12-31\"}}}}]}}".format(year)
        options = "&o={{\"per_page\":10000,\"page\":{}}}".format(page)

        full_url = base_url + query + field_list + options
#         print(full_url)
        r = requests.get(full_url)
        print(r)
        data = r.json()
        print(year,"-",page,"-",data['count'])

        print(data)
        break
        # several columns in the JSON are nested, which will create a nested dataframe.
        # This creates different dataframes for each nested object, which can all be combined later
        df = pd.io.json.json_normalize(data['patents'])
        top_level = pd.concat([top_level, df], ignore_index=True)

        
        df = pd.io.json.json_normalize(data['patents'], record_path=['applications'], meta=['patent_number'])
        applications_level = pd.concat([applications_level, df], ignore_index=True)
        
        df = pd.io.json.json_normalize(data['patents'], record_path=['assignees'], meta=['patent_number'])
        assignee_level = pd.concat([assignee_level, df], ignore_index=True)
        
        df = pd.io.json.json_normalize(data['patents'], record_path = ['cited_patents'], meta=['patent_number'])
        cited_patents_level = pd.concat([cited_patents_level, df], ignore_index=True)
        
        df = pd.io.json.json_normalize(data['patents'], record_path=['inventors'], meta=['patent_number'])
        inventor_level = pd.concat([inventor_level, df], ignore_index=True)
        
        df = pd.io.json.json_normalize(data['patents'], record_path=['foreign_priority'], meta=['patent_number'])
        foreign_priority_level = pd.concat([foreign_priority_level, df], ignore_index=True)
        
        df = pd.io.json.json_normalize(data['patents'], record_path = ['uspcs'], meta=['patent_number'])
        uspcs_level = pd.concat([uspcs_level, df], ignore_index=True)


        
        # decide if to continue to next year or next page
        if data['count'] < 10000:
            more_patents = False
        else:
            page += 1
            
      


Some of the data isnt available via the API, so it has to be extracted from the raw data downloaded from PatentsView website.
This includes the number of non-patent citations and number of figures

In [ ]:
figures = pd.read_csv('figures.tsv',delimiter='\t',usecols=['patent_id','num_figures'])
figures.rename(index=str, columns={"patent_id":"patent_number"}, inplace=True)
figures.head()

In [ ]:
otherrefs = pd.read_csv('otherreference.tsv', delimiter='\t', usecols=['uuid','patent_id'], engine='python', error_bad_lines=False)
otherrefs = otherrefs['patent_id'].value_counts().reset_index().rename(index=str, columns={'index':'patent_number','patent_id':'non-pat_refs'})
otherrefs.head()

In [ ]:
application_citations = pd.read_csv('usapplicationcitation.tsv', delimiter = '\t', usecols=['patent_id', 'date'])

application_citations.head()

In [ ]:
application_citations[application_citations['patent_id'].str.match('D703103')]

In [ ]:
master = pd.read_csv('master.csv')

In [ ]:
master[master['patent_number'].str.match('D703103')]

In [ ]:
related = pd.read_csv('usreldoc.tsv', delimiter = '\t')

related.head()

In [ ]:
related[related['doctype'].str.contains('related')]

In [ ]:
related['doctype'].unique()

In [ ]:
print("top level: ",top_level.shape,
      "applications: ",applications_level.shape,
      "assignee: ", assignee_level.shape,
      "cited patents: ",cited_patents_level.shape,
      "inventors: ", inventor_level.shape,
      "foreign priority: ", foreign_priority_level.shape,
      "classes: ", uspcs_level.shape)

# Data cleaning and combining

Now that we have all the basic data that we are interested in, it's time to clean them up into neat pretty rows for useful analysis

In [ ]:
master = pd.DataFrame()

Let's look at the top level, non-nested, level data

In [ ]:
top_level.head()

In [ ]:
master['patent_number'] = top_level['patent_number']
master.head()

In [ ]:
master['grant_year'] = top_level['patent_year']
master.head()

Extracting application date

In [ ]:
applications_level.head()

In [ ]:
applications_level['app_date'] = pd.to_datetime(applications_level['app_date'], errors='coerce').apply(lambda x: x.year)
master = pd.merge(master, applications_level, how='left', on='patent_number')

In [ ]:
master = master.rename(index=str, columns={"app_date":"app_year"}).drop(labels=['app_id'], axis=1)
master.head()

Number of inventors

In [ ]:
inventor_level.head()

In [ ]:
number_inventors = inventor_level['patent_number'].value_counts().reset_index().rename(index=str, columns={'index':'patent_number','patent_number':'num_inventors'})
master = pd.merge(master, number_inventors, how='left',on='patent_number')
master.head()

If US-first inventor

In [ ]:
master = pd.merge(master, top_level[['patent_firstnamed_inventor_country', 'patent_number']], how='left', on='patent_number')
master.head()

In [ ]:
master['patent_firstnamed_inventor_countrymaster'] = master['patent_firstnamed_inventor_country'].str.match('US').astype(float)
master.rename(index=str, columns={'patent_firstnamed_inventor_country':'us_inventor'}, inplace=True)
master.head()

In [ ]:
master.drop(labels=['us_inventor'], axis=1, inplace=True)
master.rename(index=str, columns={'patent_firstnamed_inventor_countrymaster':'us_inventor'}, inplace=True)
master.head()

any foreign patents cited

In [ ]:
master = pd.merge(master, top_level[['patent_num_foreign_citations', 'patent_number']], how='left', on='patent_number')
master['patent_num_foreign_citations'] = (master['patent_num_foreign_citations'].astype(float) > 0).astype(float)
master.rename(index=str, columns={'patent_num_foreign_citations':'cite_foreign_patent'}, inplace=True)
master.head()

Any missing patents

In [ ]:
cited_patents_level.head()

In [ ]:
missing_cits = cited_patents_level['patent_number'].value_counts().reset_index().rename(index=str, columns={'index':'patent_number','patent_number':'num_cits_reported'})
missing_cits.head()

In [ ]:
missing_cits = pd.merge(missing_cits, top_level[['patent_number','patent_num_us_patent_citations']], how='inner', on='patent_number')
missing_cits['reported_less_than_count'] = (missing_cits['num_cits_reported'] < missing_cits['patent_num_us_patent_citations'].astype(int))
missing_cits.head()

In [ ]:
any_null_cites = cited_patents_level.groupby('patent_number')['cited_patent_number'].apply(lambda x: x.isnull().any()).to_frame()
any_null_cites.reset_index(inplace=True)
any_null_cites.head()

In [ ]:
missing_cits = pd.merge(missing_cits, any_null_cites, how='inner', on='patent_number')
missing_cits.head()

In [ ]:
missing_cits['is_missing'] = (missing_cits['reported_less_than_count'] | missing_cits['cited_patent_number']).astype(int)
missing_cits.head()

In [ ]:
master = pd.merge(master, missing_cits[['patent_number','is_missing']], on='patent_number', how='left')
master.head()

In [ ]:
# master = master.drop(labels=['is_missing_x'], axis=1).rename(index=str, columns={'is_missing_y':'is_missing'})
# master.head()

Number of cited design patents

In [ ]:
cited_patents_level.head()

In [ ]:
cited_design = cited_patents_level.groupby('patent_number')['cited_patent_number'].apply(lambda x: np.count_nonzero(x.str.contains('D\d{6}'))).reset_index().rename(index=str, columns={'cited_patent_number':'num_design_cited'})
cited_design.head()

In [ ]:
master = pd.merge(master, cited_design, how='left', on='patent_number')
master.head()

Number of cited utility patents

In [ ]:
cited_utility = cited_patents_level.groupby('patent_number')['cited_patent_number'].apply(lambda x: np.count_nonzero(x.str.contains('\d{7}'))).reset_index().rename(index=str, columns={'cited_patent_number':'num_utility_cited'})
cited_utility.head()

In [ ]:
master = pd.merge(master, cited_utility, how='left', on='patent_number')
master.head()

Number of non-patent prior art

In [ ]:
otherrefs.head()

In [ ]:
master = pd.merge(master, otherrefs, on='patent_number', how='left')
master.head()

Number of figures

In [ ]:
figures.head()

In [ ]:
master = pd.merge(master, figures, on='patent_number', how='left')
master.head()

Number of assignees

In [ ]:
assignee_level.head()

In [ ]:
num_assignees = assignee_level['patent_number'].value_counts().reset_index().rename(index=str, columns={'index':'patent_number', 'patent_number':'num_assignees'})
num_assignees.head()

In [ ]:
master = pd.merge(master, num_assignees, on='patent_number', how='left')
master.head()

# Foreign Priority Date - Need to figure out

In [ ]:
foreign_priority_level.head()

In [ ]:
foreign_priority_level['patent_number'].value_counts()

In [ ]:
foreign_priority_level[foreign_priority_level['patent_number'].str.match('D665368')]

Since design patents are designated by an D in the beginning of their number, let's see if any were missclassified. If so, they need to be removed

In [36]:
#filter to make sure mis-classified patents are not included
def remove_non_design(df):
    return df[df.patent_number.str.contains('D')]
    

In [ ]:
master = remove_non_design(master)
cited_patents_level = remove_non_design(cited_patents_level)
assignee_level = remove_non_design(assignee_level)
print(master.groupby('patent_number').ngroups , cited_patents_level.groupby('patent_number').ngroups, assignee_level.groupby('patent_number').ngroups)

Now let's deal with any misssing values

In [ ]:
master.isnull().sum()

In [ ]:
master[master['us_inventor'].isnull()]

In [ ]:
#since most the N/A inventors are non-US. So they will all get a value of 0
master['us_inventor'] = master['us_inventor'].fillna(0)

In [ ]:
master[master['non-pat_refs'].isnull()]

In [ ]:
# The null non-patent reference means that no of these references were made. Fill it in with 0
master['non-pat_refs'] = master['non-pat_refs'].fillna(0)

In [ ]:
master[master['num_figures'].isnull()]

In [ ]:
#for some reason, certain patents are not listed in the figures.tsv. So we will just fill them in as 0
master['num_figures'] = master['num_figures'].fillna(0)

In [ ]:
cited_patents_level.isnull().sum()

In [ ]:
#drop the null cited patents since they can't give us any information anyways
cited_patents_level.dropna(inplace=True)

In [ ]:
assignee_level.isnull().sum()

Since assignee is a mixed bag of null, we will keep it for now and deal with it later

# There are officially 525,490 design patents represented in this dataset
Let's save the master, assignee, and cited patent so they can be used in future data analysis

In [ ]:
master.to_csv('master.csv', index=False)
cited_patents_level.to_csv('citations_patents_level.csv', index=False)
assignee_level.to_csv('assignee_level.csv', index=False)

# Priority Date
 
definition: earliest date of action for patent, which can be the application date, the foreign priority date, or dates in the related documents sections (which includes continuations)

Select the earliest year and the source of it, and the country

In [6]:
master = pd.read_csv('data/master.csv')
master.head()

,patent_number,grant_year,app_year,num_inventors,us_inventor,cite_foreign_patent,is_missing,num_design_cited,num_utility_cited,non-pat_refs,num_figures,num_assignees,priority_date
0,D257752,1981,1980,1,1.0,0.0,1,1,2,1.0,6.0,1,NaN
1,D257924,1981,1980,1,1.0,0.0,1,1,2,1.0,6.0,1,NaN
2,D258382,1981,1980,2,1.0,0.0,1,2,1,0.0,5.0,1,NaN
3,D258383,1981,1980,2,1.0,0.0,1,2,1,0.0,5.0,1,NaN
4,D258571,1981,1980,1,1.0,0.0,1,1,1,0.0,2.0,1,NaN


In [9]:
master.drop(labels=['priority_date'], axis=1, inplace=True)
master.head()

,patent_number,grant_year,app_year,num_inventors,us_inventor,cite_foreign_patent,is_missing,num_design_cited,num_utility_cited,non-pat_refs,num_figures,num_assignees
0,D257752,1981,1980,1,1.0,0.0,1,1,2,1.0,6.0,1
1,D257924,1981,1980,1,1.0,0.0,1,1,2,1.0,6.0,1
2,D258382,1981,1980,2,1.0,0.0,1,2,1,0.0,5.0,1
3,D258383,1981,1980,2,1.0,0.0,1,2,1,0.0,5.0,1
4,D258571,1981,1980,1,1.0,0.0,1,1,1,0.0,2.0,1


In [11]:
foreign = pd.read_csv('data/foreign_priority.tsv', delimiter ='\t', usecols=['patent_id','date','country_transformed'])
foreign.head()

,patent_id,date,country_transformed
0,4579440,1982-11-18,JP
1,4736480,1985-03-27,DE
2,D449111,2001-01-09,HK
3,5228311,1990-08-16,DE
4,9711226,2015-03-04,JP


In [14]:
related = pd.read_csv('data/usreldoc.tsv', delimiter='\t', usecols=['patent_id','doctype','date'])
related.head()

,patent_id,doctype,date
0,9683848,continuation-in-part,2011-12-23
1,8343191,related-publication,2008-09-04
2,9435959,related-publication,2014-05-29
3,8394761,division,0000-00-00
4,7626702,related-publication,2008-06-19


In [18]:
applications = pd.read_csv('data/application.tsv', delimiter ='\t',usecols=['patent_id','date'])
applications.head()

,patent_id,date
0,D345393,1992-12-21
1,5164715,1990-04-10
2,5177974,1988-06-23
3,5379515,1994-02-16
4,5264790,1991-07-01


In [20]:
print(foreign['patent_id'].nunique(), related['patent_id'].nunique(), applications['patent_id'].nunique())

2426215 3164930 6502933


In [22]:
foreign.rename(index=str, columns={'patent_id':'patent_number', 'date':'foreign_date'}, inplace=True)
related.rename(index=str, columns={'patent_id':'patent_number', 'date':'related_date'}, inplace=True)
applications.rename(index=str, columns={'patent_id':'patent_number', 'date':'app_date'}, inplace=True)

In [48]:
foreign['foreign_date'] = pd.to_datetime(foreign['foreign_date'], errors='coerce')
related['related_date'] = pd.to_datetime(related['related_date'], errors='coerce')
applications['app_date'] = pd.to_datetime(applications['app_date'], errors='coerce')

In [42]:
foreign = remove_non_design(foreign)
related = remove_non_design(related)
applications = remove_non_design(applications)

In [38]:
foreign.isnull().sum()

patent_number          0
foreign_date           0
country_transformed    0
dtype: int64

In [46]:
foreign_min = foreign.dropna().groupby("patent_number")["foreign_date"].apply(lambda x: x.min()).reset_index()
foreign_min.head()

,patent_number,foreign_date
0,D242583,1974-11-21
1,D242598,1975-01-15
2,D242606,1975-02-03
3,D242617,1973-10-01
4,D242626,1974-04-26


In [51]:
related.isnull().sum()

patent_number        0
doctype              0
related_date     81206
foreign_date     81206
dtype: int64

In [56]:
related_min = related.dropna().groupby('patent_number')['related_date'].apply(lambda x: x.min()).reset_index()

In [57]:
applications.isnull().sum()

patent_number    0
app_date         3
dtype: int64

In [61]:
applications.dropna(inplace=True)

In [71]:
min_date = pd.merge(foreign_min, related_min, on='patent_number', how='outer')
min_date = pd.merge(min_date, applications, on='patent_number', how='outer')
min_date.head()

,patent_number,foreign_date,related_date,app_date
0,D242583,1974-11-21,NaT,1975-05-20
1,D242598,1975-01-15,NaT,1975-07-11
2,D242606,1975-02-03,NaT,1975-07-17
3,D242617,1973-10-01,NaT,1975-09-19
4,D242626,1974-04-26,NaT,1974-08-14


In [90]:
min_date['priority_date'] = min_date.loc[:, ['foreign_date','related_date','app_date']].min(axis=1)
min_date.head()

,patent_number,foreign_date,related_date,app_date,priority_date
0,D242583,1974-11-21,NaT,1975-05-20,1974-11-21
1,D242598,1975-01-15,NaT,1975-07-11,1975-01-15
2,D242606,1975-02-03,NaT,1975-07-17,1975-02-03
3,D242617,1973-10-01,NaT,1975-09-19,1973-10-01
4,D242626,1974-04-26,NaT,1974-08-14,1974-04-26


In [91]:
min_date['priority_date'] = min_date['priority_date'].apply(lambda x: x.year)

In [104]:
master = pd.merge(master, min_date[['patent_number','priority_date']], on='patent_number', how='left')
master.head()

,patent_number,grant_year,app_year,num_inventors,us_inventor,cite_foreign_patent,is_missing,num_design_cited,num_utility_cited,non-pat_refs,num_figures,num_assignees,priority_date
0,D257752,1981,1980,1,1.0,0.0,1,1,2,1.0,6.0,1,1980
1,D257924,1981,1980,1,1.0,0.0,1,1,2,1.0,6.0,1,1980
2,D258382,1981,1980,2,1.0,0.0,1,2,1,0.0,5.0,1,1980
3,D258383,1981,1980,2,1.0,0.0,1,2,1,0.0,5.0,1,1980
4,D258571,1981,1980,1,1.0,0.0,1,1,1,0.0,2.0,1,1980


In [105]:
master.to_csv('data/master.csv', index=False)